In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd


In [2]:
orig_data = pd.read_csv("data_set/从11个特征预测学生的测试分数.csv")
X , Y = orig_data.iloc[:,0:10] , orig_data.iloc[:,-1] #切片
X = X.drop(columns=["school","classroom","student_id"],axis=1)#按列丢弃无关项
X = pd.get_dummies(X,columns=["school_setting","school_type","teaching_method","gender","lunch"],dtype=int)#独热编码 #orig_data.iloc[0:5,] X.iloc[0:5,]

一层神经网络模型  
激活函数f(x)= x  
损失函数均方误差  
优化器Adam(默认比较优秀的优化器 结合了自适应梯度Adagrad和RMSProp防梯度急剧下降的优点)

In [3]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)#Sequential模型结构 多个网络层且无多余分支的堆叠 Dense就是常用的全连接层 运算就是output = activation(dot(input, kernel)+bias) units为该层的输出维度 而非输入维度  
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)) #adam（前期降低损失迅速，随迭代次数衰减学习率） 所以可以一开始把学习率提高一点 
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) #整个数据一共遍历epoch遍 每一次拿batch_size个样本来算损失 所以一个epoch需要算epoch/batch_size遍 更新同样次数的参数
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 868us/step - loss: 633.6257
Epoch 2/100
54/54 [==============================] - 0s 532us/step - loss: 26.6278
Epoch 3/100
54/54 [==============================] - 0s 448us/step - loss: 21.5082
Epoch 4/100
54/54 [==============================] - 0s 469us/step - loss: 20.7557
Epoch 5/100
54/54 [==============================] - 0s 483us/step - loss: 19.9873
Epoch 6/100
54/54 [==============================] - 0s 567us/step - loss: 19.2612
Epoch 7/100
54/54 [==============================] - 0s 450us/step - loss: 18.4861
Epoch 8/100
54/54 [==============================] - 0s 514us/step - loss: 17.8040
Epoch 9/100
54/54 [==============================] - 0s 517us/step - loss: 17.1688
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 16.5420 - val_loss: 16.6993
Epoch 11/100
54/54 [==============================] - 0s 574us/step - loss: 16.0043
Epoch 12/100
54/54 [==============================] - 0s 564us/ste

换个优化器 随机梯度下降

In [4]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.SGD(lr=0.01,clipvalue=0.5))#梯度裁剪 Gradient Clipping +-0.5
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 733us/step - loss: 510.0822
Epoch 2/100
54/54 [==============================] - 0s 265us/step - loss: 39.8698
Epoch 3/100
54/54 [==============================] - 0s 493us/step - loss: 23.9310
Epoch 4/100
54/54 [==============================] - 0s 509us/step - loss: 21.1697
Epoch 5/100
54/54 [==============================] - 0s 496us/step - loss: 19.3262
Epoch 6/100
54/54 [==============================] - 0s 497us/step - loss: 17.8869
Epoch 7/100
54/54 [==============================] - 0s 494us/step - loss: 16.6689
Epoch 8/100
54/54 [==============================] - 0s 497us/step - loss: 15.7837
Epoch 9/100
54/54 [==============================] - 0s 555us/step - loss: 15.0186
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 14.3578 - val_loss: 14.7016
Epoch 11/100
54/54 [==============================] - 0s 460us/step - loss: 13.8656
Epoch 12/100
54/54 [==============================] - 0s 644us/ste

自适应梯度 (对初始位置要求很严格 不然直接3000遍历还有1000损失 下降平缓均匀

In [5]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adagrad(lr=0.01,epsilon=1e-6))
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 549us/step - loss: 5831.6851
Epoch 2/100
54/54 [==============================] - 0s 533us/step - loss: 5726.1411
Epoch 3/100
54/54 [==============================] - 0s 494us/step - loss: 5658.4521
Epoch 4/100
54/54 [==============================] - 0s 501us/step - loss: 5604.0244
Epoch 5/100
54/54 [==============================] - 0s 516us/step - loss: 5557.2715
Epoch 6/100
54/54 [==============================] - 0s 497us/step - loss: 5515.9189
Epoch 7/100
54/54 [==============================] - 0s 471us/step - loss: 5478.3857
Epoch 8/100
54/54 [==============================] - 0s 531us/step - loss: 5443.7070
Epoch 9/100
54/54 [==============================] - 0s 495us/step - loss: 5411.4331
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 5381.1001 - val_loss: 4474.4155
Epoch 11/100
54/54 [==============================] - 0s 721us/step - loss: 5352.5005
Epoch 12/100
54/54 [=======================

结合梯度平方的指数移动平均数来调节学习率的变化。能够在不稳定（Non-Stationary）的目标函数情况下进行很好地收敛。递归神经网络时的一个良好选择？

In [6]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-6)) #rho衰减系数 epsilon数值稳定性的小常数？
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 553us/step - loss: 8085.8545
Epoch 2/100
54/54 [==============================] - 0s 531us/step - loss: 7269.1675
Epoch 3/100
54/54 [==============================] - 0s 471us/step - loss: 6520.9087
Epoch 4/100
54/54 [==============================] - 0s 537us/step - loss: 5814.3032
Epoch 5/100
54/54 [==============================] - 0s 512us/step - loss: 5144.4268
Epoch 6/100
54/54 [==============================] - 0s 535us/step - loss: 4522.2173
Epoch 7/100
54/54 [==============================] - 0s 515us/step - loss: 3938.5032
Epoch 8/100
54/54 [==============================] - 0s 497us/step - loss: 3396.8181
Epoch 9/100
54/54 [==============================] - 0s 534us/step - loss: 2898.7349
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 2443.1765 - val_loss: 1317.0623
Epoch 11/100
54/54 [==============================] - 0s 509us/step - loss: 2031.1885
Epoch 12/100
54/54 [=======================